In [2]:
import sys;
import os

#os.chdir('experimental/fixedPointIter2/scripts')
sys.path.append("../modules/");
sys.path.append("../inputDataTraiRhizo/");
sys.path.append("../../../../CPlantBox/");
sys.path.append("../../../../CPlantBox/src")
sys.path.append("../../../build-cmake/cpp/python_binding/");


from rosi_richards10c_cyl import RichardsNCCylFoam  # C++ part (Dumux binding)

from richards import RichardsWrapper  # Python part
from richards_no_mpi import RichardsNoMPIWrapper  # Python part of cylindrcial
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import pandas as pd
import functional.van_genuchten as vg
import os
import scenario_setup
matplotlib.use('TkAgg', force=True)

In [1]:
import pickle
path = "results/results_v9.2/"
with open(path+"dataDumux10.pkl", "rb") as file:
    dataDumux10 = pickle.load(file)
#dataDumux['cc'] += a_in/100
key_changes = {'cc':'cc','getPressureHead':'htot','Cmucil':'cll','theta':'theta',
            'Viscosity':'mu_sol','initpressure':'Hp','Ks':'Ks'}

dataDumux10 = {key_changes.get(k, k): v for k, v in dataDumux10.items()}

with open(path+"dataDumux.pkl", "rb") as file:
    dataDumux = pickle.load(file)
#dataDumux['cc'] += a_in/100
key_changes = {'cc':'cc','getPressureHead':'htot','Cmucil':'cll','theta':'theta',
            'Viscosity':'mu_sol','initpressure':'Hp','Ks':'Ks'}

dataDumux = {key_changes.get(k, k): v for k, v in dataDumux.items()}


In [13]:
a_in = 0.02
names = ['cll','Hp','htot','Ks','mu_sol','theta']
dataComsol = {}
for nm in names:
    data = np.loadtxt(path+nm+".txt", delimiter=",")
    data[:,0] += a_in#/100 
    dataComsol[nm] = data
    #df = pd.read_csv(path+nm+".txt", delim_whitespace=True, header=None)


In [8]:
names = ['Hp','theta','htot','cll','Ks','mu_sol']
axesIDs = [[0, 0],[0, 1],[1, 0],[1, 1],[2, 0],[2, 1]]
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 8), sharex=True)

# First subplot (top-left): getPressureHead
for jj, nm in enumerate(names):
    aID0 = axesIDs[jj][0]
    aID1 = axesIDs[jj][1]
    for i in range(3):
        axes[aID0,aID1].plot(dataComsol[nm][:,0], dataComsol[nm][:,i+1], label=str(18000 * i)+"comsol")
        axes[aID0,aID1].plot(dataDumux['cc'], dataDumux[nm][i], label=str(18000 * i)+"dumux")
    axes[aID0,aID1].grid(True)
    axes[aID0,aID1].legend()
    axes[aID0,aID1].set_title(nm)

# Adjust layout
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [49]:
dataDumux['Ks'][0]


array([2.06374362e-16, 2.01402598e-16, 1.96617668e-16, 1.92010592e-16,
       1.87572879e-16, 1.83296503e-16, 1.79173872e-16, 1.75197813e-16,
       1.71361537e-16, 1.67658627e-16, 1.64083011e-16, 1.60628944e-16,
       1.57290988e-16, 1.54063995e-16, 1.50943088e-16, 1.47923648e-16,
       1.45001298e-16, 1.42171888e-16, 1.39431481e-16])

In [22]:
import string
from matplotlib import colors
from matplotlib.lines import Line2D
from matplotlib.ticker import ScalarFormatter
from matplotlib.transforms import ScaledTranslation

names = ['Hp','theta','htot','cll','Ks','mu_sol']
axesIDs = [[0, 0],[0, 1],[1, 0],[1, 1],[2, 0],[2, 1]]
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 8), sharex=True)
l_styles = {0:'-',1:'--',2:':'}

# Line style legend
linestyle_legend_elements = [
    Line2D([0], [0], color='black', linestyle=l_styles[0], lw=6, label=str(0)+"hr"),
    #Line2D([0], [0], color='black', linestyle=l_styles[38], lw=6, label=38),
    Line2D([0], [0], color='black', linestyle=l_styles[1], lw=6, label=str(5)+"hr"),
    Line2D([0], [0], color='black', linestyle=l_styles[2], lw=6, label=str(10)+"hr")
]

c_styles = {
    'dumux-20cells':'red',
    'dumux-10cells':'green',  # Blue
    'comsol':  'black' 
}


color_legend_elements = [
    Line2D([0], [0], color=c_styles['dumux-10cells'], lw=6, label='dumux-10cells'),
    Line2D([0], [0], color=c_styles['dumux-20cells'], lw=6, label='dumux-20cells'),
    Line2D([0], [0], color=c_styles['comsol'], lw=6, label='comsol')
]
# First subplot (top-left): getPressureHead
for jj, nm in enumerate(names):
    aID0 = axesIDs[jj][0]
    aID1 = axesIDs[jj][1]
    for i in range(3):
        axes[aID0,aID1].plot(dataComsol[nm][:,0], dataComsol[nm][:,i+1],# label=str(18000 * i)+"comsol",
                         c=c_styles["comsol"],
                         linestyle= l_styles[i]
                            )
        axes[aID0,aID1].plot(dataDumux['cc'], dataDumux[nm][i],# label=str(18000 * i)+"dumux",
                         c=c_styles["dumux-20cells"],
                         linestyle= l_styles[i])
        axes[aID0,aID1].plot(dataDumux10['cc'], dataDumux10[nm][i],# label=str(18000 * i)+"dumux",
                         c=c_styles["dumux-10cells"],
                         linestyle= l_styles[i])
    axes[aID0,aID1].grid(True)
    #axes[aID0,aID1].legend()
    axes[aID0,aID1].set_title(nm)

#axes[-1, -1].set_visible(False)
# Add legends outside the subplots
fig.legend(handles=color_legend_elements, #fontsize=15, 
           title='solver',
           #title_fontsize='20', 
           loc='center', bbox_to_anchor=(0.85, 0.25), framealpha=0)

fig.legend(handles=linestyle_legend_elements, #fontsize=15, 
           title='time',
           #title_fontsize='20', 
           loc='center', bbox_to_anchor=(0.3, 0.8), framealpha=0)
# Adjust layout
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

/tmp/ipykernel_186240/889102522.py:64: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
